**Case study comparing Akantu and SimpleRB for a loading as potentially encountered by Clearspace**

In [1]:
import sys
sys.path.append('../..')

import time
import numpy as np
import akantu as aka
from tqdm import tqdm
from wobble.rb_simple import SimpleRB
import matplotlib.pyplot as plt

In [2]:
# partly initialise the Akantu simulation to find the stable time step
start_time =  time.time()
spatial_dimension = 3
mesh_file = '../sample_data_files/mesh_files/beam.msh'

aka.parseInput('../sample_data_files/material_files/steel.dat')
mesh = aka.Mesh(spatial_dimension)
mesh.read(mesh_file)

model = aka.SolidMechanicsModel(mesh)

model.initFull(_analysis_method=aka._explicit_lumped_mass)

displacement = model.getDisplacement()
force = model.getExternalForce()

time_factor = 0.8
stable_time_step = model.getStableTimeStep() * time_factor
model.setTimeStep(stable_time_step)
max_steps = int(1 / stable_time_step)

In [3]:
# Run the SimpleRB simulation
start_time =  time.time()
time_step = stable_time_step * 100
sim=SimpleRB('SimpleRB', 3, time_step, 1,
                 mesh_file='../sample_data_files/mesh_files/beam.msh',
                 material_file='../sample_data_files/material_files/steel.dat',
                 num_modes=10,
                 force_path='../sample_data_files/force_files/case_study.txt',
                 eigenmode_path='../sample_data_files/eigenmode_files/rb_modes.csv',
                 boundary_mask= np.loadtxt('../sample_data_files/boundary_mask_files/beam_boundary_mask.txt').astype(bool))

sim.time_array = np.arange(0,1,time_step)
sim.solve()
print('Execution took ', time.time() - start_time, ' seconds')

Execution took  1.6040782928466797  seconds


In [ ]:
# Run the Akantu simulation
for row in range(force.shape[0]):
    if sim.boundary_mask[::3][row]:
        force[row] = sim.unprojected_force[:, 0].reshape(-1, 3)[row]


force_times = np.hstack((sim.force_times, 1e5))

disps=[]
j = 1
for i in tqdm(range(0, max_steps + 1)): 
    if i * stable_time_step >= force_times[j]:
        offset = 0
        for row in range(force.shape[0]):
            if sim.boundary_mask[::3][row]:
                force[row] = sim.unprojected_force[:, j].reshape(-1, 3)[row + offset]
            else:
                offset += 1
            
        j += 1
    
    model.solveStep()
    if i % 100 == 0:
        disp=model.getDisplacement()
        disps.append(disp.copy())

print('Execution took ', time.time() - start_time, ' seconds')        
        
disps = np.array(disps)

 56%|█████▌    | 64561/115558 [01:43<01:12, 701.65it/s]

In [ ]:
# Plotting parameters
SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 22

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"

In [ ]:
# Plot the displacements for the [0,0,0] corner node of the beam
fig, axs = plt.subplots(1, 3)
axes = ['x', 'y', 'z']
for i in range(3):
    axs[i].plot(sim.time_array, sim.total_displacement_vectors[i] * 100, label='SimpleRB')
    axs[i].plot(np.linspace(0,1,len(disps)), disps[:, 0, i] * 100, label='Akantu')
    axs[i].set_title(f'Displacement in {axes[i]}', fontweight='bold')

    if i == 0:
        axs[i].set_ylabel('Displacement [cm]')
        axs[i].legend()
        
    if i == 1:
        axs[i].set_xlabel('Time [s]')
    
fig.suptitle('Case Study Forcing', fontweight='bold')
fig.set_size_inches(20.5, 5.5)
fig.tight_layout()
fig.show()

In [ ]:
# Compute the RMS error and the Akantu RMS displacement
disps2 = disps.reshape(-1, 1737).T[sim.boundary_mask]
diff = sim.total_displacement_vectors[:, :] - disps2
rmse = np.sqrt(np.sum(diff ** 2, axis=0) / diff.shape[0])
divider = np.sqrt(np.sum(disps2 ** 2, axis=0) / sim.total_displacement_vectors.shape[0])
divider[divider == 0] = 1

In [ ]:
# Plot the relative RMS error
plt.plot(sim.time_array[:], rmse  / divider * 100)
plt.xlabel('Time [s]')
plt.ylabel('Error [%]')
plt.title('RMS Error as Percentage of RMS Displacement', fontweight='bold')
fig.set_size_inches(20.5, 5.5)
plt.tight_layout()